# Carga de librerías

# Carga dataset

In [1]:
# Download data from repository and unzip
!wget https://github.com/Laboratorios-2-MIT-MC/IA-Ciber/raw/main/Pr%C3%A1cticas/Proyecto%20Final/Dataset/trafic_data.zip.001 -O trafic_data.zip.001
!wget https://github.com/Laboratorios-2-MIT-MC/IA-Ciber/raw/main/Pr%C3%A1cticas/Proyecto%20Final/Dataset/trafic_data.zip.002 -O trafic_data.zip.002
!unzip -o trafic_data.zip.001

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


# Carga del modelo

In [ ]:
# Download the model and unzip
!wget https://github.com/Laboratorios-2-MIT-MC/IA-Ciber/raw/main/Pr%C3%A1cticas/Proyecto%20Final/Modelo/yolov8x.zip.001 -O yolov8x.zip.001
!wget https://github.com/Laboratorios-2-MIT-MC/IA-Ciber/raw/main/Pr%C3%A1cticas/Proyecto%20Final/Modelo/yolov8x.zip.002 -O yolov8x.zip.002
!unzip -o yolov8x.zip.001